In [75]:
# import all the libraries needed to build a toxicity classifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import itertools
import random
from typing import List, Tuple
import re

import vowpal_wabbit_next as vw
import random
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import json
from collections import defaultdict  


In [76]:
# pip install -U pip setuptools wheel
# pip install -U 'spacy[apple]'
# python -m spacy download en_core_web_sm
# https://spacy.io/usage

import spacy
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp("This is a and sentence.")

In [172]:
# [(w.text, w.pos_) for w in doc])

# Remove stop words
l = []
for w in doc:
    print(w.text, w.pos_)
    if w.pos_ == "Aux" or w.pos_ == "DET" or w.pos_ == "ADP" or w.pos_ == "PUNCT" or w.pos_ == "PRON" or w.pos_ == "CCONJ" or w.pos_ == "SCONJ" or w.pos_ == "PART" or w.pos_ == "NUM" or w.pos_ == "SYM" or w.pos_ == "CCONJ":
        # skip the word
        continue
    else:
        l.append(w.text)
print(l)

This PRON
is AUX
a PRON
and CCONJ
sentence NOUN
. PUNCT
['is', 'sentence']


In [173]:
chat_log = pd.read_json("chat_log_20230825_zackrawrr.json")
chat_log.head()

,username,chat_message,timestamp
0,captainborat2,Baldur's Gate 3 Romance Just Isn't Interesting...,2023-08-25 13:18:10.912280
1,lore_sound,Halo 2,2023-08-25 13:18:10.968594
2,juanmajfry,xdd,2023-08-25 13:18:11.384643
3,w3btree,@zackrawrr i got over diablo 4 by playing lost...,2023-08-25 13:18:11.384786
4,steeltarkus,based,2023-08-25 13:18:11.420514


In [174]:
toxicity_dataset = pd.read_json("social_media_toxicity_dataset.json")
toxicity_dataset.head()

,worker_id,task_id,task_response_id,text,Is this text toxic?
0,4RHRV9MACQGW,ccd0a1da-0e6a-453e-a6d1-6418cc4a5546,3d66075c-623e-4e64-93ac-123d379d28f0,I came here to say this exactly!,Not Toxic
1,R7XYZ4FKMFF9,d977c704-4ae3-4381-99de-95014ee57a05,acbd5847-698e-4098-9499-d547948a2da2,Thank you :),Not Toxic
2,MNTYRY6PYPQF,89a4c206-dfbc-40e8-ac73-9bb1b5c8e56e,2def7071-3e27-4b63-bd66-99f041363e2a,I feel a subreddit being born,Not Toxic
3,DR6XNZMT9KRH,41add9eb-bbe3-47d9-8ea2-b571f96fe655,b6e11a16-686d-43da-87f4-8201d8bb2238,Disturbing wholesomeness should be a thing,Not Toxic
4,G6VJRCCGZ9D6,883f5bd6-058c-460f-8e06-c0793af441d1,7f0bdd8f-428a-4ecb-b432-8e6b1f3706c0,Smite jinx... int or pentakill? Both.,Not Toxic


In [175]:
# Preprocess the toxicity dataset

# Lowercase all the text and remove all punctuation and special characters.
toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: x.lower())
toxicity_dataset['text'] = toxicity_dataset['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [176]:
# Use spacy to remove stop words
toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: nlp(x))
toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: [w.text for w in x if w.pos_ != "Aux" and w.pos_ != "DET" and w.pos_ != "ADP" and w.pos_ != "PUNCT" and w.pos_ != "PRON" and w.pos_ != "CCONJ"])
toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: ' '.join(x))

# Remove all the empty text
toxicity_dataset = toxicity_dataset[toxicity_dataset['text'] != '']


In [177]:
toxicity_dataset.head()

,worker_id,task_id,task_response_id,text,Is this text toxic?
0,4RHRV9MACQGW,ccd0a1da-0e6a-453e-a6d1-6418cc4a5546,3d66075c-623e-4e64-93ac-123d379d28f0,came here to say exactly,Not Toxic
1,R7XYZ4FKMFF9,d977c704-4ae3-4381-99de-95014ee57a05,acbd5847-698e-4098-9499-d547948a2da2,thank,Not Toxic
2,MNTYRY6PYPQF,89a4c206-dfbc-40e8-ac73-9bb1b5c8e56e,2def7071-3e27-4b63-bd66-99f041363e2a,feel subreddit being born,Not Toxic
3,DR6XNZMT9KRH,41add9eb-bbe3-47d9-8ea2-b571f96fe655,b6e11a16-686d-43da-87f4-8201d8bb2238,disturbing wholesomeness should be thing,Not Toxic
4,G6VJRCCGZ9D6,883f5bd6-058c-460f-8e06-c0793af441d1,7f0bdd8f-428a-4ecb-b432-8e6b1f3706c0,smite jinx int pentakill,Not Toxic


-----------------------------------------------

In [201]:
toxicity_dataset = pd.read_json("social_media_toxicity_dataset.json")
toxicity_dataset.head()

,worker_id,task_id,task_response_id,text,Is this text toxic?
0,4RHRV9MACQGW,ccd0a1da-0e6a-453e-a6d1-6418cc4a5546,3d66075c-623e-4e64-93ac-123d379d28f0,I came here to say this exactly!,Not Toxic
1,R7XYZ4FKMFF9,d977c704-4ae3-4381-99de-95014ee57a05,acbd5847-698e-4098-9499-d547948a2da2,Thank you :),Not Toxic
2,MNTYRY6PYPQF,89a4c206-dfbc-40e8-ac73-9bb1b5c8e56e,2def7071-3e27-4b63-bd66-99f041363e2a,I feel a subreddit being born,Not Toxic
3,DR6XNZMT9KRH,41add9eb-bbe3-47d9-8ea2-b571f96fe655,b6e11a16-686d-43da-87f4-8201d8bb2238,Disturbing wholesomeness should be a thing,Not Toxic
4,G6VJRCCGZ9D6,883f5bd6-058c-460f-8e06-c0793af441d1,7f0bdd8f-428a-4ecb-b432-8e6b1f3706c0,Smite jinx... int or pentakill? Both.,Not Toxic


In [202]:
def remove_stopwords(sentence):
    """ Remove stop words from a sentence """
    doc = nlp(sentence)
    filtered_sentence = " ".join([token.text for token in doc if not token.is_stop])
    return filtered_sentence

In [203]:
def preprocess_toxicity_dataset_text(dataframe):
    # Lowercase all the text
    dataframe['text'] = dataframe['text'].apply(lambda x: x.lower())
    
    # Remove stop words
    dataframe['text'] = dataframe['text'].apply(lambda x: remove_stopwords(x))
    
    # Remove all punctuation and special characters
    dataframe['text'] = dataframe['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
    
    # Remove all numbers unless they are 69 or 420
    dataframe['text'] = dataframe['text'].apply(lambda x: re.sub(r'\b(?!69\b|420\b)\d+\b', '', x))
    
    # Remove all words that are less than 3 characters long
    dataframe['text'] = dataframe['text'].apply(lambda x: re.sub(r'\b\w{1,3}\b', '', x))
    
    # Uncomment if you want to remove single alphabets
    # dataframe['text'] = dataframe['text'].apply(lambda x: re.sub(r'\b[a-zA-Z]\b', '', x))
    
    # Remove spaces that are more than 1
    dataframe['text'] = dataframe['text'].apply(lambda x: re.sub(' +', ' ', x))
    
    # Remove empty text entries
    dataframe = dataframe[dataframe['text'] != '']
    
    return dataframe



# # Lowercase all the text and remove all punctuation and special characters.
# toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: x.lower())

# # use lambda function to remove stop words, use the function above
# toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: remove_stopwords(x))

# # Remove all punctuation and special characters
# toxicity_dataset['text'] = toxicity_dataset['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


# # remove all numbers unless it is 69 or 420
# toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: re.sub(r'\b(?!69\b|420\b)\d+\b', '', x))

# # remove all the all the text that are less than 3 characters
# toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: re.sub(r'\b\w{1,3}\b', '', x))

# # # remove alphabets 
# # toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: re.sub(r'\b[a-zA-Z]\b', '', x))

# # remove all the empty text after removing stop words (spaces that are more than 1)
# toxicity_dataset['text'] = toxicity_dataset['text'].apply(lambda x: re.sub(' +', ' ', x))

# # remove all the empty text
# toxicity_dataset = toxicity_dataset[toxicity_dataset['text'] != '']


In [204]:
toxicity_dataset = pd.read_json("social_media_toxicity_dataset.json")
preprocessed_dataset_toxicity = preprocess_text(toxicity_dataset)
preprocessed_dataset_toxicity.head()


,worker_id,task_id,task_response_id,text,Is this text toxic?
0,4RHRV9MACQGW,ccd0a1da-0e6a-453e-a6d1-6418cc4a5546,3d66075c-623e-4e64-93ac-123d379d28f0,came exactly,Not Toxic
1,R7XYZ4FKMFF9,d977c704-4ae3-4381-99de-95014ee57a05,acbd5847-698e-4098-9499-d547948a2da2,thank,Not Toxic
2,MNTYRY6PYPQF,89a4c206-dfbc-40e8-ac73-9bb1b5c8e56e,2def7071-3e27-4b63-bd66-99f041363e2a,feel subreddit born,Not Toxic
3,DR6XNZMT9KRH,41add9eb-bbe3-47d9-8ea2-b571f96fe655,b6e11a16-686d-43da-87f4-8201d8bb2238,disturbing wholesomeness thing,Not Toxic
4,G6VJRCCGZ9D6,883f5bd6-058c-460f-8e06-c0793af441d1,7f0bdd8f-428a-4ecb-b432-8e6b1f3706c0,smite jinx pentakill,Not Toxic


In [207]:
# convert the toxicity dataset into VW format

def get_vw_example_from_row(row):
    """Converts a row of the toxicity dataset into VW format."""
    # skip the row if the text is empty
    if row["text"] == "":
        return ""
    label = '-1' if row["Is this text toxic?"] == "Toxic" else '1'
    return "{} |text {}\n".format(label, row["text"].strip())

def convert_training_data_to_vw_format(data):
    """Converts the entire toxicity dataset into VW format."""
    return ''.join([get_vw_example_from_row(row) for _, row in data.iterrows()])

def parse_lines(parser: vw.TextFormatParser, input_str: str) -> List[vw.Example]:
    result = []
    for line in input_str.split("\n"):
        # print(line)
        result.append(line)
    return result


In [208]:
workspace = vw.Workspace()
parser = vw.TextFormatParser

# write to file all the commands
with open("train.txt", "w") as f:
    # randomize the order of the dataset
    preprocessed_dataset_toxicity = preprocessed_dataset_toxicity.sample(frac=1).reset_index(drop=True)
    for command in parse_lines(parser, convert_training_data_to_vw_format(preprocessed_dataset_toxicity)):
        f.write(command + "\n")

-------------------------------------------------

In [233]:
# preprocess the chat log dataset for testing 

def preprocess_chat_message(dataframe):
    # Lowercase all the chat_message
    dataframe['chat_message'] = dataframe['chat_message'].apply(lambda x: x.lower())
    
    # Remove stop words
    dataframe['chat_message'] = dataframe['chat_message'].apply(lambda x: remove_stopwords(x))
    
    # Remove all punctuation and special characters
    dataframe['chat_message'] = dataframe['chat_message'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
    
    # Remove all numbers unless they are 69 or 420
    dataframe['chat_message'] = dataframe['chat_message'].apply(lambda x: re.sub(r'\b(?!69\b|420\b)\d+\b', '', x))
    
    # Remove all words that are less than 3 characters long
    dataframe['chat_message'] = dataframe['chat_message'].apply(lambda x: re.sub(r'\b\w{1,3}\b', '', x))
    
    # Uncomment if you want to remove single alphabets
    # dataframe['chat_message'] = dataframe['chat_message'].apply(lambda x: re.sub(r'\b[a-zA-Z]\b', '', x))
    
    # Remove spaces that are more than 1
    dataframe['chat_message'] = dataframe['chat_message'].apply(lambda x: re.sub(' +', ' ', x))
    
    # Remove empty chat_message entries
    dataframe = dataframe[dataframe['chat_message'] != '']
    
    return dataframe


In [239]:
def convert_chat_message_to_vw_format(chat_messages):
    for message in chat_messages:
        yield f"|text {message}"

chat_log = pd.read_json("chat_log_20230825_zackrawrr.json")
preprocessed_dataset_chat_log = preprocess_chat_message(chat_log)

# Get the chat message column
chat_messages = preprocessed_dataset_chat_log['chat_message']

# Write to file all the commands
with open("test.txt", "w") as f:
    for command in convert_chat_message_to_vw_format(chat_messages):
        f.write(command + "\n")